# QDET Compute Module Tutorial

This notebook demonstrates all available tools in the QDET compute module. The compute module provides quantum circuit compilation, optimization, error mitigation, resource management, and distributed processing capabilities for quantum computing workflows.

## 1. Import Required Libraries

Import necessary libraries including Qiskit, pandas, numpy, and all tools from the compute module.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sys
import os
sys.path.append(os.path.abspath('..'))

# Import Qiskit components
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Import compute module tools
from qudet.compute import (
    BackendManager,
    CircuitOptimizer,
    QuantumCircuitCompiler,
    QuantumCircuitOptimizer,
    QuantumNativeGateTranspiler,
    QuantumErrorMitigation,
    QuantumNoiseModel,
    QuantumCalibrationalAnalyzer,
    QuantumResourceAllocator,
    QuantumPriorityScheduler,
    QuantumCostEstimator,
    DistributedQuantumProcessor,
    HardwareLayoutSelector
)

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries and compute tools imported successfully!")

## 2. Load and Explore the Iris Dataset

Load the iris.csv dataset and examine its structure for use in our compute demonstrations.

In [ ]:
# Load the Iris dataset
iris_df = pd.read_csv('../qudet/datasets/iris.csv')

print("Dataset Shape:", iris_df.shape)
print("\nFirst 5 rows:")
print(iris_df.head())

print("\nDataset Summary:")
print(iris_df.describe())

# For compute demonstrations, we'll use normalized feature values
X = iris_df.iloc[:, :-1].values
X_norm = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

print(f"\nNormalized feature shape: {X_norm.shape}")
print(f"Feature range: [{X_norm.min():.2f}, {X_norm.max():.2f}]")

## 3. Backend Manager

**Description**: Centralized controller for quantum hardware connections. Handles switching between local simulation and cloud QPU providers (IBM, IonQ). Provides graceful fallback to simulators if real quantum hardware is unavailable.

**Use Case**: Connect to quantum backends and manage execution resources.

In [ ]:
# Get simulator backend
backend = BackendManager.get_backend("simulator")

print("Backend Manager Configuration:")
print(f"Backend name: {backend.name if hasattr(backend, 'name') else 'AerSimulator'}")
print(f"Backend type: {type(backend).__name__}")
print(f"Available: ✓")

# Get recommended optimization level for this backend
opt_level = BackendManager.optimize_level("simulator")
print(f"Recommended optimization level: {opt_level}")

## 4. Circuit Optimizer

**Description**: Optimizes quantum circuits to reduce depth and execution cost. Removes redundant gates and applies optimization passes to minimize gate count while preserving quantum computation results.

**Use Case**: Reduce circuit complexity and lower quantum hardware costs.

In [ ]:
# Create a sample quantum circuit with redundancy
qr = QuantumRegister(4, 'q')
cr = ClassicalRegister(4, 'c')
qc = QuantumCircuit(qr, cr)

# Add some gates with redundancy (H-H cancellation)
for i in range(4):
    qc.h(i)
    qc.h(i)  # This cancels the previous H gate

# Add some other gates
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 3)

qc.measure(range(4), range(4))

print("Original Circuit:")
print(f"Depth: {qc.depth()}")
print(f"Number of gates: {qc.size()}")

# Optimize the circuit
optimizer = CircuitOptimizer(level=3)
optimized_qc = optimizer.optimize(qc)

print("\nOptimized Circuit:")
print(f"Depth: {optimized_qc.depth()}")
print(f"Number of gates: {optimized_qc.size()}")
print(f"Gate reduction: {((qc.size() - optimized_qc.size()) / qc.size() * 100):.1f}%")

## 5. Quantum Circuit Compiler

**Description**: Compiles abstract quantum algorithms to native gate sets. Decomposes high-level gate operations into device-specific native gates that hardware can execute.

**Use Case**: Compile quantum algorithms for different quantum hardware platforms (IBM, Rigetti, IonQ).

In [ ]:
# Initialize Quantum Circuit Compiler for IBM native gates
compiler = QuantumCircuitCompiler(target_gateset="ibm", optimization_level=2)

# Create a circuit specification with abstract gates
circuit_spec = {
    'gates': [
        {'type': 'h', 'qubits': [0]},
        {'type': 'x', 'qubits': [1]},
        {'type': 'y', 'qubits': [2]},
        {'type': 'z', 'qubits': [3]},
        {'type': 'cx', 'qubits': [0, 1]}
    ],
    'qubits': 4
}

# Compile to native IBM gates (RX, RZ, CX)
native_gates = ['rx', 'rz', 'cx']
compiled = compiler.compile(circuit_spec, native_gates)

print("Quantum Circuit Compilation Results:")
print(f"Target gate set: {compiler.target_gateset.upper()}")
print(f"Original gates: {compiled['gates'][:3]}")
print(f"Circuit qubits: {compiled['qubits']}")
print(f"Circuit depth: {compiled['depth']}")
print(f"Compilation stats: {compiler.get_compilation_stats()}")

## 6. Hardware Layout Selector

**Description**: Selects the best physical qubits on a quantum device based on error rates. Avoids noisy qubits and places quantum data on "golden qubits" with lowest error rates.

**Use Case**: Improve circuit accuracy by using the best-performing qubits on real quantum hardware.

In [ ]:
# Use the simulator backend for layout selection
layout_selector = HardwareLayoutSelector(backend)

# Select best qubits for a 4-qubit circuit
n_qubits_needed = 4
best_qubits = layout_selector.find_best_subgraph(n_qubits_needed)

print("Hardware Layout Selection Results:")
print(f"Qubits needed: {n_qubits_needed}")
print(f"Best qubits selected: {best_qubits}")
print(f"Layout optimization: ✓ Complete")

# In a real scenario with a physical backend, this would select qubits
# based on actual device error rates and connectivity constraints
print("\nNote: On real hardware (IBM, IonQ), this selects physically connected")
print("qubits with the lowest readout error rates.")

## 7. Quantum Error Mitigation

**Description**: Mitigates errors in quantum circuit execution using techniques like zero-noise extrapolation. Calibrates on test circuits and applies corrections to noisy results from real quantum hardware.

**Use Case**: Improve accuracy of results from noisy quantum devices.

In [ ]:
# Initialize Error Mitigation with Zero-Noise Extrapolation
qem = QuantumErrorMitigation(mitigation_method="zero_noise_extrapolation")

# Create test circuits for calibration
test_circuits = [{'type': 'test_1'}, {'type': 'test_2'}, {'type': 'test_3'}]

# Simulate noisy results from quantum hardware
noisy_results = np.array([0.7, 0.65, 0.72, 0.68, 0.75])  # Errors present

# Calibrate on test data
qem.calibrate(test_circuits, noisy_results)

# Apply error mitigation
mitigated_results = qem.mitigate(noisy_results)

print("Quantum Error Mitigation Results:")
print(f"Mitigation method: {qem.mitigation_method}")
print(f"Test circuits used: {qem.calibration_data['test_circuits']}")
print(f"Estimated error rate: {qem.calibration_data['error_rate']:.4f}")
print(f"\nNoisy results (avg): {noisy_results.mean():.4f}")
print(f"Mitigated results (avg): {mitigated_results.mean():.4f}")
print(f"Improvement: {((mitigated_results.mean() - noisy_results.mean()) / noisy_results.mean() * 100):.1f}%")

## 8. Quantum Noise Model

**Description**: Defines and applies quantum noise models to simulate realistic quantum hardware effects. Models various noise types (depolarizing, amplitude damping, phase damping) for circuit simulation.

**Use Case**: Test circuit robustness against realistic hardware noise before quantum execution.

In [ ]:
# Create noise models for different quantum hardware characteristics
noise_types = ["depolarizing", "amplitude_damping", "phase_damping"]
error_rates = [0.01, 0.005, 0.002]  # 1%, 0.5%, 0.2%

print("Quantum Noise Models:")
print("-" * 50)

for noise_type, error_rate in zip(noise_types, error_rates):
    nm = QuantumNoiseModel(noise_type=noise_type, error_rate=error_rate)
    
    # Create a sample quantum state
    state = np.array([1/np.sqrt(2), 1/np.sqrt(2)])  # Bell state
    
    # Apply noise
    noisy_state = nm.apply_noise(state)
    
    print(f"\n{noise_type.replace('_', ' ').title()}:")
    print(f"  Error rate: {error_rate*100:.2f}%")
    print(f"  Original state fidelity: 1.000")
    print(f"  Noisy state fidelity: {np.abs(np.dot(np.conj(state), noisy_state)):.4f}")

print("\n" + "-" * 50)
print("These noise models help predict real hardware performance")

## 9. Quantum Resource Allocator

**Description**: Allocates and manages quantum computing resources including qubits and circuit depth limits. Tracks resource usage across multiple tasks and prevents resource conflicts.

**Use Case**: Manage quantum resource allocation for multiple concurrent quantum computing tasks.

In [ ]:
# Initialize Resource Allocator for a 127-qubit system (like IBM)
qra = QuantumResourceAllocator(total_qubits=127, max_circuit_depth=1000)

# Allocate qubits for different tasks
task1_qubits = qra.allocate_qubits("task_1", n_qubits=4)
task2_qubits = qra.allocate_qubits("task_2", n_qubits=6)
task3_qubits = qra.allocate_qubits("task_3", n_qubits=3)

print("Quantum Resource Allocation Results:")
print(f"Total available qubits: {qra.total_qubits}")
print(f"Max circuit depth: {qra.max_circuit_depth}")

print(f"\nTask 1: {len(task1_qubits)} qubits allocated -> {task1_qubits}")
print(f"Task 2: {len(task2_qubits)} qubits allocated -> {task2_qubits}")
print(f"Task 3: {len(task3_qubits)} qubits allocated -> {task3_qubits}")

# Update resource usage
qra.update_resource_usage("task_1", depth=150, n_gates=45)
qra.update_resource_usage("task_2", depth=200, n_gates=60)

# Get resource summary
summary = qra.get_resource_summary()
print(f"\nTotal qubits allocated: {summary['total_qubits_allocated']}")
print(f"Qubits utilization: {summary['qubit_utilization_percent']:.1f}%")
print(f"Active tasks: {summary['n_tasks']}")

## 10. Quantum Priority Scheduler

**Description**: Schedules quantum computing jobs with priority levels. Manages job queuing, prioritization, and execution order based on importance and resource requirements.

**Use Case**: Schedule multiple quantum jobs with different priorities for fair resource distribution.

In [ ]:
# Initialize Quantum Priority Scheduler
scheduler = QuantumPriorityScheduler(max_queue_size=100)

# Submit jobs with different priorities
jobs = [
    {"job_id": "high_priority_1", "priority": 1, "qubits": 4, "depth": 150},
    {"job_id": "normal_job_1", "priority": 2, "qubits": 3, "depth": 100},
    {"job_id": "normal_job_2", "priority": 2, "qubits": 5, "depth": 120},
    {"job_id": "low_priority_1", "priority": 3, "qubits": 2, "depth": 80},
]

for job in jobs:
    scheduler.submit_job(job)

# Get execution order
execution_order = scheduler.get_execution_order()

print("Quantum Priority Scheduler Results:")
print(f"Total jobs queued: {len(jobs)}")
print(f"Queue size limit: {scheduler.max_queue_size}")

print(f"\nExecution Order (by priority):")
for idx, job_id in enumerate(execution_order, 1):
    job = next(j for j in jobs if j['job_id'] == job_id)
    print(f"  {idx}. {job_id} (priority: {job['priority']}, qubits: {job['qubits']})")

## 11. Quantum Cost Estimator

**Description**: Estimates the cost of quantum circuit execution on cloud quantum platforms. Calculates costs based on circuit depth, qubit usage, and platform-specific pricing.

**Use Case**: Budget quantum computing expenses and optimize circuits for cost-efficiency.

In [ ]:
# Initialize Cost Estimator
cost_estimator = QuantumCostEstimator(provider="ibm")

# Estimate costs for different circuit configurations
circuits = [
    {"name": "Small Circuit", "n_qubits": 2, "depth": 50, "shots": 1024},
    {"name": "Medium Circuit", "n_qubits": 5, "depth": 200, "shots": 1024},
    {"name": "Large Circuit", "n_qubits": 10, "depth": 500, "shots": 1024},
]

print("Quantum Cost Estimation Results (IBM):")
print("-" * 60)

for circuit in circuits:
    cost = cost_estimator.estimate_cost(circuit)
    print(f"\n{circuit['name']}:")
    print(f"  Qubits: {circuit['n_qubits']}, Depth: {circuit['depth']}, Shots: {circuit['shots']}")
    print(f"  Estimated cost: ${cost['total_cost']:.2f}")
    print(f"    - Per-qubit cost: ${cost.get('per_qubit_cost', 0):.4f}")
    print(f"    - Per-gate cost: ${cost.get('per_gate_cost', 0):.4f}")
    print(f"    - Per-shot cost: ${cost.get('per_shot_cost', 0):.6f}")

print("\n" + "-" * 60)
print("Note: Actual costs vary by provider and current pricing")

## 12. Distributed Quantum Processor

**Description**: Manages parallel execution of quantum encoding using Dask. Distributes large datasets across multiple CPUs for concurrent quantum circuit generation before sending to quantum hardware.

**Use Case**: Process large-scale datasets in parallel for efficient quantum computation.

In [ ]:
# For distributed processing, we'll show a simulated example
# Create a sample encoder-like object
from qudet.encoders.rotation import RotationEncoder

# Initialize a distributed quantum processor
# Note: Full distributed processing requires Dask installation
try:
    processor = DistributedQuantumProcessor(
        encoder=RotationEncoder(n_qubits=4),
        n_workers=2
    )
    
    # Process a subset of iris data
    iris_sample = iris_df.iloc[:10, :-1].copy()
    
    print("Distributed Quantum Processor Results:")
    print(f"Encoder qubits: {processor.encoder.n_qubits}")
    print(f"Number of workers: {processor.n_workers}")
    print(f"Input data shape: {iris_sample.shape}")
    
    # Process the data
    circuits = processor.process_large_dataset(iris_sample)
    
    print(f"Circuits generated: {len(circuits)}")
    print(f"✓ Distributed processing completed")
    
except Exception as e:
    print(f"Distributed processing demonstration (Dask optional):")
    print(f"Would process {len(iris_df.iloc[:10, :-1])} rows in parallel")
    print(f"Actual error: {str(e)[:80]}")

## Summary

This tutorial demonstrated all major tools in the QDET compute module:

1. **BackendManager** - Connects to quantum hardware (simulators and real QPUs) with graceful fallback
2. **CircuitOptimizer** - Reduces circuit depth and gate count for cost-efficient execution
3. **QuantumCircuitCompiler** - Compiles abstract algorithms to native gate sets
4. **HardwareLayoutSelector** - Selects best physical qubits based on error rates
5. **QuantumErrorMitigation** - Mitigates errors using zero-noise extrapolation techniques
6. **QuantumNoiseModel** - Simulates realistic quantum hardware noise for testing
7. **QuantumResourceAllocator** - Manages qubit and circuit depth allocation across tasks
8. **QuantumPriorityScheduler** - Schedules quantum jobs with priority-based queuing
9. **QuantumCostEstimator** - Estimates execution costs on cloud quantum platforms
10. **DistributedQuantumProcessor** - Processes large datasets in parallel using Dask

These tools provide a complete infrastructure for managing, optimizing, and executing quantum circuits at scale, from local simulation to distributed cloud execution on real quantum hardware.